In [1]:
%pip install pyikarus --no-build-isolation --verbose --upgrade

Using pip 23.2.1 from /dune/dune-common/build-cmake/dune-env/lib/python3.10/site-packages/pip (python 3.10)
  Using cached pyikarus-0.3.3.tar.gz (741 kB)
  Preparing metadata (pyproject.toml) ...   Running command Preparing metadata (pyproject.toml)
  Note: dune-common will get version requirement (<= 2.9.0) in pyproject.toml as stated in Python-Requires.
  Note: dune-grid will get version requirement (<= 2.9.0) in pyproject.toml as stated in Python-Requires.
  Note: dune-geometry will get version requirement (<= 2.9.0) in pyproject.toml as stated in Python-Requires.
  Note: dune-istl will get version requirement (<= 2.9.0) in pyproject.toml as stated in Python-Requires.
  running dist_info
  creating /tmp/pip-modern-metadata-0kb7q1h1/pyikarus.egg-info
  writing /tmp/pip-modern-metadata-0kb7q1h1/pyikarus.egg-info/PKG-INFO
  writing dependency_links to /tmp/pip-modern-metadata-0kb7q1h1/pyikarus.egg-info/dependency_links.txt
  writing requirements to /tmp/pip-modern-metadata-0kb7q1h1/pyi

In [ ]:
%pip install dune-iga --no-build-isolation --verbose --upgrade

Using pip 23.2.1 from /dune/dune-common/build-cmake/dune-env/lib/python3.10/site-packages/pip (python 3.10)
Note: you may need to restart the kernel to use updated packages.


### Import all packages

In [ ]:
import matplotlib
import dune.grid
import dune.functions
# import pyikarus as iks
# import pyikarus.finite_elements
# import pyikarus.utils
# import pyikarus.assembler
# import pyikarus.dirichletValues
import numpy as np
import scipy as sp
from dune.vtk import  vtkWriter
from dune.iga import reader as readeriga
from dune.iga.basis import defaultGlobalBasis , Nurbs
from dune.iga import IGAGrid

In [ ]:
reader = (readeriga.json, "element_trim_xb.ibra")
gridView = IGAGrid(reader, dimgrid=2, dimworld=2)

DUNE-INFO: Generating dune-py module in /dune/dune-common/build-cmake/dune-env/.cache/dune-py
DUNE-INFO: Compiling ReferenceElements (new)


ImportError: initialization failed

### Create grid

In [ ]:
help(gridView)
gridView.plot()

### Add Lagrangian basis

In [ ]:
basisLagrange1 = dune.functions.defaultGlobalBasis(grid, dune.functions.Power(dune.functions.Lagrange(order=2),2))
print('We have {} dofs.'.format(len(basisLagrange1)))
print('We have {} vertices.'.format(grid.size(2)))
print('We have {} elements.'.format(grid.size(0)))

### Init load factor and displacement vector

In [ ]:
d = np.zeros(len(basisLagrange1))
lambdaLoad = iks.ValueWrapper(.1)

### Create finite element requirements

In [ ]:
req= iks.FErequirements()
req.addAffordance(iks.ScalarAffordances.mechanicalPotentialEnergy)
req.addAffordance(iks.VectorAffordances.forces)
req.addAffordance(iks.MatrixAffordances.stiffness)

req.insertParameter(iks.FEParameter.loadfactor,lambdaLoad)
req.insertGlobalSolution(iks.FESolutions.displacement,d)

### Define volume load and boundary loads

In [ ]:
def volumeLoad(x,lambdaVal) :
    return np.array([lambdaVal*x[0]*2*0, 2*lambdaVal*x[1]*0])

def neumannLoad(x,lambdaVal) :
    return np.array([lambdaVal*100, lambdaVal])

neumannVertices = np.zeros(len(basisLagrange1), dtype=bool)
basisLagrange1.interpolate(neumannVertices, lambda x :  True  if x[1]==1 else False)

boundaryPatch = iks.utils.boundaryPatch(grid,neumannVertices)

### Create vector of finite elements

In [ ]:
fes = []
for e in grid.elements:
    fes.append(iks.finite_elements.linearElasticElement(basisLagrange1,e,1000,0.2,volumeLoad,boundaryPatch,neumannLoad))


### print forces and stiffness of first element

In [ ]:
forces = np.zeros(18)
stiffness = np.zeros((18,18))
fes[1].calculateVector(req,forces)
fes[1].calculateMatrix(req,stiffness)
np.set_printoptions(precision=3)
print('Forces:\n {}'.format(forces))
print('Stiffness:\n {}'.format(stiffness))
print('Eigenvalues: ',np.real(sp.linalg.eigvals(stiffness)))

### Create Dirichlet boundary conditions

In [ ]:
dirichletValues = iks.dirichletValues(basisLagrange1) 

def fixBottomEdge(vec,localIndex,localView,intersection):
    if (intersection.geometry.center[1]==-1):
        vec[localView.index(localIndex)]= True

dirichletValues.fixBoundaryDOFsUsingLocalViewAndIntersection(fixBottomEdge)

### Create assembler

In [ ]:
assembler = iks.assembler.sparseFlatAssembler(fes,dirichletValues)
#assemblerDense = iks.assembler.denseFlatAssembler(fes,dirichletValues)

Msparse = assembler.getMatrix(req)
forces = assembler.getVector(req)
#print(Msparse)
#print(forces)

### Solve for displacements and write to paraview

In [ ]:
d = sp.sparse.linalg.spsolve(Msparse,-forces)
dispFunc = basisLagrange1.asFunction(d)



In [ ]:
writer = vtkWriter( grid, "nameTest",  pointData   = {( "displacement",(0,1)):dispFunc})

### Plot here using matplot lib

In [ ]:
import dune.plotting
dune.plotting.plot(solution=dispFunc,gridLines=None)

In [ ]:
from helper import plotDeformedGrid
plotDeformedGrid(grid,dispFunc) 